In [1]:
#!pip install bs4 lxml datasets transformers accelerate evaluate

In [2]:
import requests
from bs4 import BeautifulSoup as soup
import datasets
import numpy as np
import evaluate
from evaluate import evaluator
from transformers import AutoTokenizer, DataCollatorForLanguageModeling, AutoModelForCausalLM,DataCollatorForSeq2Seq, GPT2Tokenizer, GPT2Model, AutoModelForSequenceClassification,AutoModelForSeq2SeqLM, DataCollatorWithPadding, TrainingArguments, Trainer, pipeline, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import Dataset, load_dataset, DatasetDict
import torch
import matplotlib.pyplot as plt
from random import randint
from time import sleep
import re
import csv

In [3]:
urls_juan = [
    {
    'url': 'https://www.tripadvisor.es/Hotel_Review-g150800-d154643-Reviews-Hotel_Estoril-Mexico_City_Central_Mexico_and_Gulf_Coast.html',
    'max_or_num': 205
  },
    {
        'url': 'https://www.tripadvisor.es/Hotel_Review-g150800-d1147625-Reviews-Hampton_Inn_Suites_Mexico_City_Centro_Historico-Mexico_City_Central_Mexico_and_Gulf_Co.html',
        'max_or_num': 2875
    },
    {

    'url': 'https://www.tripadvisor.es/Hotel_Review-g150800-d3533088-Reviews-Gamma_Ciudad_de_Mexico_Santa_Fe-Mexico_City_Central_Mexico_and_Gulf_Coast.html',
    'max_or_num': 160
    },

    {
        'url': 'https://www.tripadvisor.es/Hotel_Review-g150800-d154654-Reviews-Pedregal_Palace_Hotel-Mexico_City_Central_Mexico_and_Gulf_Coast.html',
        'max_or_num': 65
    },

    {
        'url': 'https://www.tripadvisor.es/Hotel_Review-g150800-d5998759-Reviews-City_Express_Plus_Patio_Universidad-Mexico_City_Central_Mexico_and_Gulf_Coast.html',
        'max_or_num': 815
    },

    {
        'url': 'https://www.tripadvisor.es/Hotel_Review-g150800-d596936-Reviews-We_Hotel_Aeropuerto-Mexico_City_Central_Mexico_and_Gulf_Coast.html',
        'max_or_num': 595
    },

    {
        'url': 'https://www.tripadvisor.es/Hotel_Review-g150800-d152472-Reviews-Boutique_Hotel_de_Cortes-Mexico_City_Central_Mexico_and_Gulf_Coast.html',
        'max_or_num': 150
    },
    {
        'url': 'https://www.tripadvisor.es/Hotel_Review-g150782-d2366469-Reviews-IStay_Hotel_Monterrey_Historico-Monterrey_Northern_Mexico.html',
        'max_or_num': 2045
    }

    ]

In [4]:
urls_laura = [

      {
          'url': 'https://www.tripadvisor.es/Hotel_Review-g147289-d5960333-Reviews-Embassy_Suites_By_Hilton_Santo_Domingo-Santo_Domingo_Santo_Domingo_Province_Dominican_.html',
          'max_or_num': 1935
      },
      {
          'url': 'https://www.tripadvisor.es/Hotel_Review-g147289-d1627198-Reviews-Hotel_Luis_V-Santo_Domingo_Santo_Domingo_Province_Dominican_Republic.html',
          'max_or_num': 30
      },

      {
          'url': 'https://www.tripadvisor.es/Hotel_Review-g147289-d1118058-Reviews-Hotel_Discovery-Santo_Domingo_Santo_Domingo_Province_Dominican_Republic.html',
          'max_or_num': 70,
      },
      {
           'url': 'https://www.tripadvisor.es/Hotel_Review-g147289-d296113-Reviews-Hotel_Dona_Elvira-Santo_Domingo_Santo_Domingo_Province_Dominican_Republic.html',
          'max_or_num': 180
      },
      {
          'url': 'https://www.tripadvisor.es/Hotel_Review-g147289-d6610577-Reviews-Hotel_Jade-Santo_Domingo_Santo_Domingo_Province_Dominican_Republic.html',
          'max_or_num': 55
      },

      {
          'url': 'https://www.tripadvisor.es/Hotel_Review-g147289-d1985995-Reviews-Ramada_by_Wyndham_Princess_Santo_Domingo-Santo_Domingo_Santo_Domingo_Province_Dominica.html',
          'max_or_num': 135
      },

      {
          'url': 'https://www.tripadvisor.es/Hotel_Review-g147289-d588434-Reviews-Napolitano_Hotel-Santo_Domingo_Santo_Domingo_Province_Dominican_Republic.html',
          'max_or_num': 265
      }
  ]

In [5]:
urls_luis = [
    {
        'url': 'https://www.tripadvisor.es/Hotel_Review-g294480-d594688-Reviews-Dreams_Playa_Bonita_Panama-Panama_City_Panama_Province.html',
        'max_or_num': 2360
    },

    {
        'url': 'https://www.tripadvisor.es/Hotel_Review-g294480-d300127-Reviews-Sheraton_Grand_Panama-Panama_City_Panama_Province.html',
        'max_or_num': 910
    },
    {
        'url': 'https://www.tripadvisor.es/Hotel_Review-g294480-d4136565-Reviews-Hotel_San_Remo-Panama_City_Panama_Province.html',
        'max_or_num': 50
    },
    {
        'url': 'https://www.tripadvisor.es/Hotel_Review-g294480-d301871-Reviews-Hospedium_Princess_Hotel_Panama-Panama_City_Panama_Province.html',
        'max_or_num': 620
    },
    {
        'url': 'https://www.tripadvisor.es/Hotel_Review-g294480-d305436-Reviews-El_Panama_by_Faranda_Grand_Hotel-Panama_City_Panama_Province.html',
        'max_or_num': 1330
    },

    {
        'url': 'https://www.tripadvisor.es/Hotel_Review-g294480-d4108807-Reviews-Grand_International_Hotel-Panama_City_Panama_Province.html',
        'max_or_num': 765
    },
    {
        'url': 'https://www.tripadvisor.es/Hotel_Review-g294480-d305497-Reviews-Crowne_Plaza_Panama-Panama_City_Panama_Province.html',
        'max_or_num': 710
    }

]

In [6]:
urls_eve = [
    {
        'url': 'https://www.tripadvisor.es/Hotel_Review-g147320-d242220-Reviews-Best_Western_Plus_Condado_Palm_Inn-San_Juan_Puerto_Rico.html',
        'max_or_num': 2215,

    },
    {
        'url': 'https://www.tripadvisor.es/Hotel_Review-g147320-d150496-Reviews-Sheraton_Old_San_Juan_Hotel-San_Juan_Puerto_Rico.html',
        'max_or_num': 2880
    },
    {
        'url': 'https://www.tripadvisor.es/Hotel_Review-g147320-d150443-Reviews-The_Condado_Plaza_Hilton-San_Juan_Puerto_Rico.html',
        'max_or_num': 5635
    },
    {
        'url': 'https://www.tripadvisor.es/Hotel_Review-g147320-d150242-Reviews-Canario_Boutique_Hotel-San_Juan_Puerto_Rico.html',
        'max_or_num': 265
    },
    {
        'url': 'https://www.tripadvisor.es/Hotel_Review-g147320-d150243-Reviews-Sandy_Beach_Hotel-San_Juan_Puerto_Rico.html',
        'max_or_num': 370
    },
    {
        'url': 'https://www.tripadvisor.es/Hotel_Review-g147320-d549750-Reviews-S_J_Suites_Hotel-San_Juan_Puerto_Rico.html',
        'max_or_num': 305
    }
]


In [9]:
urls_rafa = [
    {
        'url': 'https://www.tripadvisor.es/Hotel_Review-g309293-d531394-Reviews-Hotel_Inca_Real-San_Jose_San_Jose_Metro_Province_of_San_Jose.html',
        'max_or_num': 155,

    },
    {
        'url': 'https://www.tripadvisor.es/Hotel_Review-g150798-d3317896-Reviews-Aurora_Suites-Guadalajara_Guadalajara_Metropolitan_Area.html?spAttributionToken=MjExNjI4NTE',
        'max_or_num': 35
    },
    {
        'url': 'https://www.tripadvisor.es/Hotel_Review-g150798-d3248695-Reviews-One_Guadalajara_Centro-Guadalajara_Guadalajara_Metropolitan_Area.html',
        'max_or_num': 910
    },
    {
        'url': 'https://www.tripadvisor.es/Hotel_Review-g150798-d1757741-Reviews-Best_Western_Plus_Gran_Hotel_Centro_Historico-Guadalajara_Guadalajara_Metropolitan_Are.html',
        'max_or_num': 290
    },
    {
        'url': 'https://www.tripadvisor.es/Hotel_Review-g150798-d154509-Reviews-Hotel_Fenix-Guadalajara_Guadalajara_Metropolitan_Area.html',
        'max_or_num': 165
    },
    {
        'url': 'https://www.tripadvisor.es/Hotel_Review-g150798-d155695-Reviews-Camino_Real_Guadalajara-Guadalajara_Guadalajara_Metropolitan_Area.html',
        'max_or_num': 500
    },
    {
        'url': 'https://www.tripadvisor.es/Hotel_Review-g150798-d154515-Reviews-Hotel_Las_Pergolas-Guadalajara_Guadalajara_Metropolitan_Area.html',
        'max_or_num': 50
    },
    {
        'url': 'https://www.tripadvisor.es/Hotel_Review-g150798-d514738-Reviews-Hotel_Aranzazu_ECO-Guadalajara_Guadalajara_Metropolitan_Area.html',
        'max_or_num': 130
    },
    {
        'url': 'https://www.tripadvisor.es/Hotel_Review-g150798-d629909-Reviews-Real_Inn_Guadalajara_Expo-Guadalajara_Guadalajara_Metropolitan_Area.html',
        'max_or_num': 165
    },
]

In [10]:
#Colocar aqui el diccionario que te corresponde
urls = urls_rafa

In [11]:
reviews_x_pages = 5

def get_links(url_dict):

  links = []
  url = url_dict['url']
  max_or_num = url_dict['max_or_num']  # El valor de max_or_num es el mayor múltiplo de 5 menor al total de reseñas en español.
                 # Como en este caso, al momento de hacer el web-scraping se tenían 53 comentarios en español,
                 # el valor de max_or_num sería 50.


  #reviews_x_pages = 5    # tienes que verificar si van de 10 en 10, o 5 en 5 las reseñas en cada página, porque cada caso es diferente....

  for k in range(0, max_or_num, reviews_x_pages):
    a = url
    if k == 0:
      ss = '{}'     # Para la primera página es el -or con valor vacío.
    else:
      ss = str(k)  # convertimos a string el valor numérico.
    a = a[:(a.find('Reviews')+7)] + '-or' + ss + a[(a.find('Reviews')+7):]  # Los índices de "Reviews" van de 0 a 6, ya que tiene 7 letras.
    links.append(a)

  return links

In [12]:

def get_reviews(url_dict):
  reviews = []
  stars =[]


  numpages = int( url_dict['max_or_num'] / reviews_x_pages )                    # puede tardar varios minutos si son muchos registros a extraer.

  for i,link in enumerate(links[0:numpages]):
    print(f'enlace: {link} \n página: {i + 1} de {numpages}')

    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36' }
    html2 = requests.get(link, headers=headers)
    #print(html2.status_code)

    sleep(randint(1,5))   # se incluyen intervalos de sleep para que no se estén haciendo llamadas de manera continua a la página
                          # y que al detectar esta actividad continua, se pueda detectar como una actividad sospechosa por el servidor
                          # y nos pueda bloquear de manera automática por ello.
    bsobj2 = soup(html2.content,'lxml')   # objeto de la librería BeautifulSoup para llevar a cabo nuestra tarea.

    # Extraemos los comentarios de cualquiera de las siguientes dos formas:
    # Incluimos algunos Print solo para visualizar la manera en que se lleva a cabo el proceso:

    #for review in bsobj2.findAll('span', class_='QewHA'):       # Forma 1:  Estamos en "span", Observa el guión-bajo al final de "class_"
    for review in bsobj2.findAll('span', {'class' : 'QewHA'}):   # Forma 2
        #print(review.text)
        reviews.append(review.text.strip())

    sleep(randint(1,5))
    bsobj3 = soup(html2.content,'lxml')

    # Extraigamos también las calificaciones (estrellas):
    # Observa que las calificaciones están en unidades de 10, 20, 30, 40, 50.
    # En particular en este caso haremos la extracción de todo el string donde se encuentra la
    # calificación al hotel por parte del usuario y a partir de ahí mediante regex puedes
    # fácilmente realizar la extracción de la calificación:
    for cal in bsobj3.findAll('div', {'class' : 'Hlmiy'}):
      #  Aquí puedes incluir tu regex para extraer solo la calificación numérica.
      stars.append(cal)

  return (reviews, stars)

In [13]:
import math

def turn_stars_into_ratings(ratings):

  threshold = 0.8
  max_rating = 50
  labels = []

  for rating in ratings:
      rating = int(rating)/max_rating

      if rating < (math.floor(rating) + threshold):
          label =  math.floor(rating)

      else:
          label = math.ceil(rating)

      labels.append(label)

  return labels

In [14]:
for url in urls:


  html = requests.get(url['url'],
                    headers={"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}
                    )
  print(html.status_code)

  links = get_links(url)
  reviews, stars = get_reviews(url)

  ratings = [re.search(r'\d+', rating.findChild('span')['class'][1]).group() for rating in stars]

  labels = turn_stars_into_ratings(ratings)

  review_dict = {'reviews': reviews, 'labels': labels}

  ds = datasets.Dataset.from_dict(review_dict)
  ds.to_pandas().to_csv('./tripadvisor_comments_train.csv', mode = 'a', index = False, encoding = 'utf8', quoting=csv.QUOTE_NONNUMERIC)




200
enlace: https://www.tripadvisor.es/Hotel_Review-g309293-d531394-Reviews-or{}-Hotel_Inca_Real-San_Jose_San_Jose_Metro_Province_of_San_Jose.html 
 página: 1 de 31
enlace: https://www.tripadvisor.es/Hotel_Review-g309293-d531394-Reviews-or5-Hotel_Inca_Real-San_Jose_San_Jose_Metro_Province_of_San_Jose.html 
 página: 2 de 31
enlace: https://www.tripadvisor.es/Hotel_Review-g309293-d531394-Reviews-or10-Hotel_Inca_Real-San_Jose_San_Jose_Metro_Province_of_San_Jose.html 
 página: 3 de 31
enlace: https://www.tripadvisor.es/Hotel_Review-g309293-d531394-Reviews-or15-Hotel_Inca_Real-San_Jose_San_Jose_Metro_Province_of_San_Jose.html 
 página: 4 de 31
enlace: https://www.tripadvisor.es/Hotel_Review-g309293-d531394-Reviews-or20-Hotel_Inca_Real-San_Jose_San_Jose_Metro_Province_of_San_Jose.html 
 página: 5 de 31
enlace: https://www.tripadvisor.es/Hotel_Review-g309293-d531394-Reviews-or25-Hotel_Inca_Real-San_Jose_San_Jose_Metro_Province_of_San_Jose.html 
 página: 6 de 31
enlace: https://www.tripadvisor